# Imports

In [1]:
# Imports
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False #disable ugly white lines which are present in google colab for matplotlib
import numpy as np
import datetime
today = datetime.datetime.today() #To work with datetime values. Only relative time matters in this project, so selecting a random date is not a problem. 
from sklearn.metrics import classification_report,auc,r2_score,matthews_corrcoef
import shap
from catboost import CatBoostClassifier,CatBoostRegressor,Pool
from catboost.utils import get_roc_curve
import xgboost as xgb
import pandas as pd
import re
from tqdm import tqdm
import seaborn as sns
from tabulate import tabulate
from scipy.stats import linregress,ttest_ind,ranksums
import scipy.stats as st
pd.options.display.float_format = '{:20,.2f}'.format
np.set_printoptions(suppress=True)
from numpy.random import RandomState

amsterdam_data_path = "../ESICM/atrial_fibrillation_causality/data/amsterdam/"

# Reading data

## Setting up the global vars

In [2]:
RS_multiple = 42
np.random.seed(RS_multiple)

causal_time_window = 6
PEEP_setting = 5

causal  = "NOR" 
dataset_source = "AMST"

causal_label = "is_given_Noradrenaline (Norepinefrine)_causal"
    
causal_string_df = causal

## Features, pre-defined confounders, and unnecessary columns

In [3]:
with_adm_drop_columns = ["dateofdeath_delta","admittedat_delta","origin",
              "lengthofstay","destination","weightgroup","agegroup","dateofdeath",
              "admittedat","heightgroup","specialty","dateofdeath_delta","admittedat_delta","weightsource","dischargedat","heightsource",
              "gender","Mortality","AF_orig","AF_measuredat","AF","new_onset_AF","Preadmission_AF","patientid","location","admissionyeargroup"]



drop_columns = ["dateofdeath_delta","admittedat_delta","admissionid","origin",
              "lengthofstay","destination","weightgroup","agegroup","dateofdeath",
              "admittedat","heightgroup","specialty","dateofdeath_delta","admittedat_delta","weightsource","dischargedat","heightsource",
              "gender","Mortality","AF_orig","AF_measuredat","AF","new_onset_AF","Preadmission_AF","patientid","location","admissionyeargroup"]


feature_columns = ['Age', 
        'max_Hartfrequentie', 
        'min_CVD',
        'mean_O2 concentratie (Set)', 
        'urgency', 
        'mean_UrineCAD',
        'is_given_Furosemide (Lasix)', 
        'fluid_balance', 
        'slope_ABP gemiddeld',
        'mean_O2-Saturatie (bloed)', 
        'max_Fosfaat (bloed)',
        'max_ABP systolisch',
        'min_Lactaat (bloed)'
    ]


feature_columns.append('mean_PEEP (Set)')

obj_causal_cols =[
'sepsis_bool',
'is_given_Propofol (Diprivan)']


causality_dataframe_columns=[
    "causal",
    "distr_change",
    "combo",
    "balance_test",
    "balance_train",
    "NO AF T1 patients",
    "AF T1 patients",
    "NO AF T0 patients",
    "AF T0 patients",
    "propensity_before",
    "propensity_after",
    "iterations",
    "CATE","p-value"
]

amsterdam_causality_dataframe = pd.DataFrame(columns=causality_dataframe_columns)

## Reading data

In [4]:
AF_dataset = pd.read_csv(amsterdam_data_path+"AF_dataset_"+str(causal_time_window)+"_hours.csv")
causal_dataset = pd.read_csv(amsterdam_data_path+causal+"_causal_AF_dataset_causal_window_"+str(causal_time_window)+"_hours.csv")

causal_dataset["is_given_Noradrenaline (Norepinefrine)"]=0
causal_dataset.loc[causal_dataset['mean_Noradrenaline (Norepinefrine)']>0,"is_given_Noradrenaline (Norepinefrine)"]=1

causal_dataset = causal_dataset.rename(columns={'is_given_Noradrenaline (Norepinefrine)':'is_given_Noradrenaline (Norepinefrine)_causal'})
causal_dataset.loc[causal_dataset['is_given_Noradrenaline (Norepinefrine)_causal'].isna(),'is_given_Noradrenaline (Norepinefrine)_causal'] = 0


causal_dataset = causal_dataset[causal_dataset.columns.drop(with_adm_drop_columns)]

AF_dataset = AF_dataset[AF_dataset.columns.difference(causal_dataset.columns.drop("admissionid"))].merge(causal_dataset,how='left',on='admissionid')
AF_dataset.loc[AF_dataset['mean_PEEP (Set)'].isna(),'mean_PEEP (Set)'] = 0

AF_dataset.loc[AF_dataset.Age==65,'Age'] = 69
AF_dataset.loc[AF_dataset.Age==35,'Age'] = 39
AF_dataset.loc[AF_dataset.Age==45,'Age'] = 49
AF_dataset.loc[AF_dataset.Age==55,'Age'] = 59
AF_dataset.loc[AF_dataset.Age==79,'Age'] = 79
AF_dataset.loc[AF_dataset.Age==85,'Age'] = 80

AF_dataset.loc[AF_dataset['is_given_Noradrenaline (Norepinefrine)'].isna(),'is_given_Noradrenaline (Norepinefrine)'] = 0

AF_dataset.loc[AF_dataset['is_given_Noradrenaline (Norepinefrine)_causal'].isna(),'is_given_Noradrenaline (Norepinefrine)_causal'] = 0

## Preprocessing datasets

In [5]:
add_zero_columns = [
            "is_given_Noradrenaline (Norepinefrine)",
            "is_given_Calcium Glubionaat (Calcium Sandoz)", 
            "is_given_Dopamine (Inotropin)", 
            "cardiac_surg_bool", 
            "is_given_Magnesiumsulfaat (MgSO4)"
            "is_given_Propofol (Diprivan)", 
            "is_given_Furosemide (Lasix)", 
            "is_given_Fentanyl", 
            "is_given_LoopDiuretics",
]

for column in add_zero_columns:
    if column not in AF_dataset.columns.values:
        AF_dataset[column] = 0

AF_dataset.loc[AF_dataset['mean_Magnesiumsulfaat (MgSO4)']
                > 0, "is_given_Magnesiumsulfaat (MgSO4)"] = 1

AF_dataset.loc[AF_dataset['mean_Calcium Glubionaat (Calcium Sandoz)']
                > 0, "is_given_Calcium Glubionaat (Calcium Sandoz)"] = 1

AF_dataset.loc[(AF_dataset['mean_Furosemide (Lasix)'] > 0),
                "is_given_LoopDiuretics"] = 1

AF_dataset.loc[AF_dataset['mean_Dopamine (Inotropin)']
                > 0, 'is_given_Dopamine (Inotropin)'] = 1

AF_dataset.loc[AF_dataset['mean_Noradrenaline (Norepinefrine)']
                > 0, 'is_given_Noradrenaline (Norepinefrine)'] = 1

In [7]:
AF_dataset.loc[AF_dataset['mean_PEEP (Set)'].isna(),'mean_PEEP (Set)'] = 0

## Train test split

In [8]:
from sklearn.model_selection import train_test_split

AF_1_df = AF_dataset[AF_dataset.AF==1]
AF_0_matched_df = AF_dataset[(AF_dataset.AF==0)&(AF_dataset.admissionid!=AF_dataset.date_corresponds_to_AF_admid)]

train_admissionid,test_admissionid = train_test_split(AF_1_df.admissionid.unique(), test_size=0.33, random_state=42)

train_AF_1_dataset = AF_1_df[AF_1_df.admissionid.isin(train_admissionid)]
test_AF_1_dataset = AF_1_df[AF_1_df.admissionid.isin(test_admissionid)]

train_AF_0_dataset = AF_0_matched_df[AF_0_matched_df.date_corresponds_to_AF_admid.isin(train_AF_1_dataset.admissionid)]
test_AF_0_dataset = AF_0_matched_df[AF_0_matched_df.date_corresponds_to_AF_admid.isin(test_AF_1_dataset.admissionid)]

train_AF_dataset = train_AF_0_dataset.append(train_AF_1_dataset).reset_index(drop=True).sample(len(train_AF_0_dataset)+len(train_AF_1_dataset),random_state=42)
test_AF_dataset = test_AF_0_dataset.append(test_AF_1_dataset).reset_index(drop=True).sample(len(test_AF_0_dataset)+len(test_AF_1_dataset),random_state=42)

# Function definitions

In [ ]:
RS = 42

def NOR_dataset(TX,dataset,undersample_AF):
     return pd.concat([dataset[(dataset['is_given_Noradrenaline (Norepinefrine)_causal']==TX)
                                       &(dataset.AF==undersample_AF)],dataset[(dataset['is_given_Noradrenaline (Norepinefrine)_causal']==TX)
                                                                                         &(dataset.AF==1-undersample_AF)].sample(len(dataset[(dataset['is_given_Noradrenaline (Norepinefrine)_causal']==TX)
                                                                                                                                               &(dataset.AF==undersample_AF)]),random_state=RS)])                                                                                                                        &(dataset.AF==undersample_AF)]),random_state=RS)])
def undersample_check(causal,TX,dataset): 
    if causal == "NOR":
        return len(dataset[(dataset['is_given_Noradrenaline (Norepinefrine)_causal']==TX)&(dataset.AF==1)])<len(dataset[(dataset['is_given_Noradrenaline (Norepinefrine)_causal']==TX)&(dataset.AF==0)])

def merge_confounder_features(causal_cols,confounder_list):
    return_cols = list(causal_cols.copy())
    for feature in confounder_list:
        if feature not in causal_cols:
            bool_passed = False
            for feature_causal in causal_cols:
                
                if len(feature.split('_', 1))>1 and (feature.split('_', 1)[1] in feature_causal):
                    bool_passed = True
                elif not len(feature.split('_', 1))>1 and feature == feature_causal:
                    bool_passed = True

            if not bool_passed:
                return_cols = return_cols + [feature]

    return return_cols

# Propensity 

## Propensity function

In [12]:
from scipy.special import logit
from sklearn.neighbors import NearestNeighbors

def propensity_preprocessing(X,target,features,index,RS=42,one_to_one_matching = False,multiple_exact=1):
    X_train_or = X[features].copy(deep=True)
    y_train_or = X[target]
    
    if y_train_or.sum()/len(y_train_or)<0.5:
        y_train = pd.concat([y_train_or[y_train_or==1],y_train_or[y_train_or==0].sample(len(y_train_or[y_train_or==1]),random_state=RS)])
        X_train = pd.concat([X_train_or[y_train_or==1],X_train_or[y_train_or==0].sample(len(y_train_or[y_train_or==1]),random_state=RS)])
    else:
        y_train = pd.concat([y_train_or[y_train_or==0],y_train_or[y_train_or==1]])
        X_train = pd.concat([X_train_or[y_train_or==0],X_train_or[y_train_or==1]])

    propensity_class_balance = [y_train.sum()/len(y_train),1-y_train.sum()/len(y_train)]
    model = CatBoostClassifier(verbose=0,n_estimators=250,class_weights=propensity_class_balance)
    model.fit(Pool(X_train,y_train))
    
    (fpr, tpr, thresholds) = get_roc_curve(model, Pool(data=X_train,label=y_train), plot=False)    
    prev_propensity_auc = auc(fpr,tpr)
    
    train_AF_dataset_match = X[features+[target]+[index]].copy(deep=True).reset_index(drop=True)
    train_AF_dataset_match["propensity_score_logit"]=0
    
    y_train_proba = model.predict_proba(train_AF_dataset_match[causal_cols])
    y_train_logit = np.array([logit(xi) for xi in y_train_proba[:,1]])
    train_AF_dataset_match.loc[:,"propensity_score_logit"]=y_train_logit
    
    NO_NOR_train_match = train_AF_dataset_match[train_AF_dataset_match[causal_label] == 0].copy(deep=True)
    NO_NOR_train_match = NO_NOR_train_match.reset_index(drop=True)
    NOR_train_match = train_AF_dataset_match[train_AF_dataset_match[causal_label] == 1].copy(deep=True)
    NOR_train_match = NOR_train_match.reset_index(drop=True)

    Neighbours = len(NO_NOR_train_match) if one_to_one_matching else multiple_exact

    knn = NearestNeighbors(n_neighbors=Neighbours)# , p = 2, radius=np.std(y_train_logit) * 0.25)
    knn.fit(NO_NOR_train_match[['propensity_score_logit']].to_numpy())

    matched_element_arr = np.zeros((len(NOR_train_match),multiple_exact))
    j = 0
    if one_to_one_matching:
        for row in NOR_train_match.iterrows():
            distance,indexes = knn.kneighbors(np.reshape([row[1].propensity_score_logit],(-1,1)),n_neighbors=Neighbours)

            for idx in indexes[0,:]:
                if idx not in matched_element_arr[:,0]:
                    matched_element_arr[j,0] = idx
                    break
            j = j+1

    else:
        for row in NOR_train_match.iterrows():
            distance,indexes = knn.kneighbors(np.reshape([row[1].propensity_score_logit],(-1,1)),n_neighbors=Neighbours)
            for i in range(multiple_exact):
                matched_element_arr[j,i] = indexes[0,i]

            j = j+1

    if one_to_one_matching:  
        all_matched_data = pd.concat([train_AF_dataset_match[train_AF_dataset_match[causal_label] == 1], NO_NOR_train_match.iloc[matched_element_arr[:,0]]])
    else:
        all_matched_data = pd.concat([train_AF_dataset_match[train_AF_dataset_match[causal_label] == 1], NO_NOR_train_match.iloc[matched_element_arr[:,0]]])
        for i in range(multiple_exact-1):
            all_matched_data = pd.concat([all_matched_data, NO_NOR_train_match.iloc[matched_element_arr[:,i]]])

    all_matched_data = all_matched_data.drop_duplicates(index)
    
    (fpr, tpr, thresholds) = get_roc_curve(model, Pool(data=all_matched_data[features],label=all_matched_data[causal_label]), plot=False)
    after_propensity_auc = auc(fpr,tpr)
    
    train_AF_dataset_temp = train_AF_dataset[train_AF_dataset[index].isin(all_matched_data[index])].copy(deep=True)
    return train_AF_dataset_temp,prev_propensity_auc,after_propensity_auc

## Train test split, feature selection, and model training

In [13]:
causal_label = "is_given_Noradrenaline (Norepinefrine)_causal"

causal_cols = list(AF_dataset.columns.values)

causal_cols =  [col for col in causal_cols if col not in drop_columns]

causal_cols = [x for x in causal_cols if not re.search("kurt|max|min|slope|causal|mean_Nor|Noradren|mean_NOR|AF",x)]

X_train_or = train_AF_dataset[causal_cols].copy(deep=True)

y_train_or = train_AF_dataset[causal_label]

y_train = y_train_or
X_train = X_train_or
                                            

X_test_or = test_AF_dataset[causal_cols].copy(deep=True)
y_test_or = test_AF_dataset[causal_label]
y_test = y_test_or
X_test = X_test_or

catboost_model = CatBoostClassifier

In [ ]:
from powershap import PowerShap

reduce_to_pure_confounders = True

propensity_class_balance = [y_train.sum()/len(y_train),1-y_train.sum()/len(y_train)]
CB_treat = catboost_model(verbose=False,iterations=500,class_weights=propensity_class_balance)

selector = PowerShap(
    model = CB_treat,
    verbose=True,
)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)
propensity_cols_confounders = X_test.columns.values

# Start causallib

In [17]:
from causallib.estimation import Standardization, StratifiedStandardization, XLearner, RLearner
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier

test_size_constant = 0.33

use_matched = False
use_confounder_features = True
use_only_confounder_features = False

if use_matched:
    train_AF_dataset_temp = train_AF_dataset
    test_AF_dataset_temp = test_all_matched_data.copy(deep=True)
    
else:
    train_AF_dataset_temp = train_AF_dataset
    test_AF_dataset_temp = test_AF_dataset

method_string = "PS prop match" if use_matched else "No matching"
method_string = method_string + " + " + ("conf feat" if use_confounder_features and use_only_confounder_features else "conf feat + feat" if use_confounder_features and not use_only_confounder_features else "feat")

if use_confounder_features:
    if use_only_confounder_features:
        causallib_cols = propensity_cols_confounders
    else:
        causallib_cols = merge_confounder_features(feature_columns,propensity_cols_confounders)
else:
    causallib_cols = feature_columns

class_balance = [train_AF_dataset_temp.AF.sum()/len(train_AF_dataset_temp),1-train_AF_dataset_temp.AF.sum()/len(train_AF_dataset_temp)]

X_causal = train_AF_dataset_temp.copy(deep=True).reset_index(drop=True)
X_causal_test = test_AF_dataset_temp.copy(deep=True).reset_index(drop=True)

# Causalteshap

In [18]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../../causalteshap/") # go to parent dir

In [20]:
X_causal_all = pd.concat([X_causal,X_causal_test])

In [22]:
from causalteshap.causalteshap import CausalteShap

S_learner = CatBoostClassifier(iterations=5000,verbose=0,class_weights=class_balance,use_best_model=True,cat_features=["T"])

causalteshap_class = CausalteShap(model=S_learner,verbose=False,significance_factor=0.04,meta_learner="S",candidate_convergence=False)
causalteshap_class.fit(X=X_causal_all[causallib_cols],T=X_causal_all[causal_label].astype(np.int32).values,y=X_causal_all["AF"].astype(np.int32))
causalteshap_class.get_analysis_df().to_csv

,ks-statistic,ttest,fligner,T1-T0,abs(T1)-abs(T0),predictive,candidate
min_CVD,0.03,0.64,0.00,-0.01,0.05,1.00,1.00
fluid_balance,0.03,0.87,0.00,-0.00,-0.07,1.00,1.00
mean_Kreatinine (bloed),0.09,0.98,0.00,-0.00,-0.03,1.00,1.00
T,0.00,0.00,0.00,0.62,0.43,1.00,1.00
min_Lactaat (bloed),2.10,0.44,0.02,-0.00,0.01,0.00,1.00
mean_pH (bloed),0.21,0.95,0.02,0.00,0.02,0.00,1.00
Age,0.00,0.69,0.82,-0.00,-0.00,0.00,0.00
max_Hartfrequentie,9.44,0.91,0.76,-0.00,-0.00,0.00,0.00
mean_O2 concentratie (Set),1.47,0.34,0.72,-0.01,-0.01,0.00,0.00
urgency,13.61,0.99,0.58,0.00,-0.01,0.00,0.00


In [28]:
causalteshap_class.get_analysis_df().to_csv("noradrenaline_results_AF_causalteshap.csv")